# Modeling with StatsModels

## 1. Ordinary Least Square
- Column Names
- Log Transformation
- Condition Number
- Standard Scaling

# 2. Dimensionality Reduction
- ANOVA
- F-test and Feature Influence

# 3. Outlier
- Cook's Distance

# 4. Regularization
- Lasso

# 5. Diagnosis of Regression
- Residual Normality Test
- Partial Regression Plot

# 6. Cross Validatoin

# 7. Test
- score

In [1]:
%matplotlib inline
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import warnings
import sys
import datetime
import scipy as sp
import statsmodels.stats.api as sms
import statsmodels.api as sm
from patsy import dmatrix
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import utils.statsmodel_helper as sh
import utils.feature_selection as fs
import utils.preprocessing as pp
import utils.error_calculator as ec


class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit()
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)

df_train_macro = pd.read_csv('../code/data/train_macro3.csv', index_col=0)
df_test_macro = pd.read_csv('../code/data/test_macro3.csv', index_col=0)

sys.setrecursionlimit(1500)

degree = 2
skewness_limit = 1
num_of_cooks = 2
num_of_f_test = 10

## Column Names

In [2]:
new_cols = []
for col in  list(df_train_macro.columns):
    col = col.replace('-', '_').replace('+', '_').replace(':', '_').replace('~', '_').replace('*', '_')
    new_cols.append('_'+col)
df_train_macro.columns = new_cols

new_cols = []
for col in list(df_test_macro.columns):
    col = col.replace('-', '_').replace('+', '_').replace(':', '_').replace('~', '_').replace('*', '_')
    new_cols.append('_'+col)
df_test_macro.columns = new_cols

categorial_ivs = list(set(df_train_macro.columns) - set(df_train_macro._get_numeric_data().columns))
numeric_ivs = df_train_macro._get_numeric_data().columns.drop('_price_doc').tolist()

## Log Transformation
Transform data with skewness greater than 1.

In [3]:
features_to_log = []
for f in df_train_macro._get_numeric_data().columns:
    skewness = sp.stats.skew(df_train_macro[f])
    if skewness > skewness_limit:
        features_to_log.append(f)

for col in df_train_macro._get_numeric_data().columns:
    if col != '_price_doc':
        min_val_train = min(df_train_macro[col])
        min_val_test  = min(df_test_macro[col])
        min_val = min(min_val_train, min_val_test)
        if min_val <= 0:
            df_train_macro[col] += (np.abs(min_val) + 0.1)
            df_test_macro[col]  += (np.abs(min_val) + 0.1)
    else:
        min_val_train = min(df_train_macro[col])
        if min_val_train <= 0:
            df_train_macro[col] += (np.abs(min_val_train) + 0.1)

In [4]:
formula = sh.make_statsmodels_ols_formula(numeric_ivs, categorial_ivs, '_price_doc', log_vs=features_to_log, degree=degree, scale=False)
model = sm.OLS.from_formula(formula, data=df_train_macro)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(_price_doc)   R-squared:                       0.421
Model:                            OLS   Adj. R-squared:                  0.420
Method:                 Least Squares   F-statistic:                     580.3
Date:                Mon, 18 Nov 2019   Prob (F-statistic):               0.00
Time:                        17:49:27   Log-Likelihood:                -19242.
No. Observations:               30404   AIC:                         3.856e+04
Df Residuals:                   30365   BIC:                         3.889e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================
                                                             coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                  0.0034      0.001      3.862      0.000       0.002       0.005
C(_railroad_terminal_raion)[T.yes]                         0.0420      0.016      2.576      0.010       0.010       0.074
C(_thermal_power_plant_raion)[T.yes]                      -0.0211      0.018     -1.167      0.243      -0.057       0.014
C(_oil_chemistry_raion)[T.yes]                            -0.0023      0.008     -0.293      0.769      -0.018       0.013
C(_water_1line)[T.yes]                                     0.0008      0.013      0.060      0.952      -0.025       0.026
C(_radiation_raion)[T.yes]                                -0.0293      0.013     -2.197      0.028      -0.055      -0.003
C(_culture_objects_top_25)[T.yes]                         -0.0100      0.019     -0.514      0.607      -0.048       0.028
C(_ecology)[T.good]                                       -0.0071      0.014     -0.507      0.612      -0.034       0.020
C(_ecology)[T.no data]                                     0.0221      0.009      2.357      0.018       0.004       0.041
C(_ecology)[T.poor]                                       -0.0022      0.013     -0.173      0.863      -0.027       0.023
C(_ecology)[T.satisfactory]                               -0.0306      0.015     -2.100      0.036      -0.059      -0.002
C(_nuclear_reactor_raion)[T.yes]                          -0.0158      0.017     -0.929      0.353      -0.049       0.018
C(_product_type)[T.OwnerOccupier]                          0.1468      0.013     11.302      0.000       0.121       0.172
C(_big_market_raion)[T.yes]                                0.0369      0.019      1.979      0.048       0.000       0.073
C(_incineration_raion)[T.yes]                             -0.0324      0.015     -2.143      0.032      -0.062      -0.003
C(_railroad_1line)[T.yes]                                 -0.0716      0.020     -3.593      0.000      -0.111      -0.033
C(_detention_facility_raion)[T.yes]                        0.0155      0.018      0.863      0.388      -0.020       0.051
C(_big_road1_1line)[T.yes]                                -0.0429      0.019     -2.304      0.021      -0.079      -0.006
np.log(_usdrub)                                            0.0122      0.001     12.338      0.000       0.010       0.014
np.log(_full_sq)                                           0.0095      0.004      2.666      0.008       0.003       0.017
np.log(_life_sq)                                           0.0129      0.008      1.714      0.087      -0.002       0.028
_floor                                                     0.0076      0.002      4.818      0.000       0.004       0.011
np.log(_

## Condition Number
Large condition number occurs when the scale of data changes significantly due to the unit difference. Scaling can decrease condition number. Multicollinearity can also cause large condition number. We can handle this by reducing dimensionality with variance inflation factor.

## Standard Scaling
Standalize variables by removing the mean and scaling to unit variance.

In [ ]:
formula = sh.make_statsmodels_ols_formula(numeric_ivs, categorial_ivs, '_price_doc', log_vs=features_to_log, degree=degree, scale=True)
model = sm.OLS.from_formula(formula, data=df_train_macro)
result = model.fit()
result.summary()

Scaling did not significantly decrease the condition number.

# 2. Dimensionality Reduction
## ANOVA

In [ ]:
anova = sm.stats.anova_lm(result, typ=2)
anova


We can remove features with p-value equal or greater than 0.05 since they have very small influences on the dependent variable

## F-test and Feature Influence

In [ ]:
result, sms_vars, formula = fs.by_f_test(df_train_macro, formula, repeat=num_of_f_test)
result.summary()

# 3. Outlier
## Cook's Distance
- Find data with large leverage and residual by calculating Cook's distance.

In [ ]:
df_train_macro_with_outliers = df_train_macro.copy(deep=True)
df_train_macro, model, result = pp.remove_outliers(df_train_macro, formula, repeat=3)
result.summary()

# 4. Regularization
## Lasso
Find variables with zero coefficient when Lasso regularization is applied.

In [ ]:
result_lasso = model.fit_regularized(alpha=0.001, L1_wt=1)

Let's remove features with zero coefficient to reduce dimensionality.

In [ ]:
sms_vars = []
for idx, coef in enumerate(result_lasso.params):
    if coef ==0:
        continue
    feature = result_lasso.params.index[idx]
    if feature == 'Intercept':
        continue
    startDelPos = feature.find('[')
    endDelPos = feature.find(']')
    feature = feature.replace(feature[startDelPos:endDelPos+1], '')
    sms_vars.append(feature)

In [ ]:
formula = 'np.log(_price_doc) ~ ' + " + ".join(sms_vars)
model =sm.OLS.from_formula(formula, data=df_train_macro)
result = model.fit()
result.summary()

# 5. Diagnosis of Regression
## Residual Normality Test

In [ ]:
# outlier remove result 
sp.stats.probplot(result.resid, plot=plt)
plt.show()

In [ ]:
test = sms.omni_normtest(result.resid)
for xi in zip(['Chi^2', 'P-value'], test):
    print("%-12s: %6.3f" % xi)

# Partial Regression Plot
Let's visualize the influence of a single independent variable.

In [ ]:
fig = plt.figure(figsize=(10,70))
sm.graphics.plot_partregress_grid(result, fig=fig)
fig.suptitle("")
plt.show()

# 6. Cross Validation

In [ ]:
dm = dmatrix(" + ".join(sms_vars) + ' + np.log(_price_doc)', df_train_macro_with_outliers, return_type="dataframe")
X = dm[dm.columns.drop(['np.log(_price_doc)'])]
y = dm['np.log(_price_doc)']
cv = cv = KFold(n_splits=1000, shuffle=True, random_state=0)
r2s = cross_val_score(SMWrapper(sm.OLS), X, y, scoring='r2', cv=cv)
r2s.mean()

In [ ]:
plt.hist(r2s, bins=100)

In [ ]:
y_pred = np.exp(result.predict(df_test_macro))
y_pred = y_pred.to_frame('price_doc')
y_pred.to_csv('./data/stats_models_{}.csv'.format(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')), header=True, index=True)

## Score

0.39773